In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input
import tensorflow_datasets as tfds

In [2]:
# Load Horses or Humans dataset
train_data, test_data = tfds.load('horses_or_humans', split=['train', 'test'], as_supervised=True)


In [3]:
# Resize and normalize the images
def preprocess_img(image, label):
    image = tf.image.resize(image, (224, 224))  # EfficientNetB0 requires 224x224 images
    image = preprocess_input(image)
    return image, label

In [4]:
train_data = train_data.map(preprocess_img)
test_data = test_data.map(preprocess_img)

In [5]:
# Load the EfficientNetB0 model pre-trained on ImageNet data
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))


16705208/16705208 [==============================] - 3s 0us/step


In [6]:
# Freeze the layers of the pre-trained model
for layer in base_model.layers:
    layer.trainable = False


In [7]:
# Create a new model on top of the pre-trained model
model = models.Sequential()
model.add(base_model)
model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dense(1, activation='sigmoid'))

In [8]:
# Compile the model
model.compile(optimizer=Adam(lr=1e-4), loss='binary_crossentropy', metrics=['accuracy'])


In [9]:
# Train the model
batch_size = 32
history = model.fit(
    train_data.batch(batch_size),
    steps_per_epoch=train_data.cardinality().numpy() // batch_size,
    epochs=10,
    validation_data=test_data.batch(batch_size),
    validation_steps=test_data.cardinality().numpy() // batch_size
)

Epoch 1/10



32/32 [==============================] - 33s 836ms/step - loss: 0.2752 - accuracy: 0.9609 - val_loss: 0.1259 - val_accuracy: 1.0000
Epoch 2/10
 1/32 [..............................] - ETA: 2s - loss: 0.1174 - accuracy: 1.0000WARNING:tensorflow:Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches (in this case, 320 batches). You may need to use the repeat() function when building your dataset.


32/32 [==============================] - 5s 163ms/step - loss: 0.1174 - accuracy: 1.0000 - val_loss: 0.1213 - val_accuracy: 1.0000


In [10]:
# Evaluate the model on the test set
test_score = model.evaluate(test_data.batch(batch_size), steps=test_data.cardinality().numpy() // batch_size)
print(f'Test loss: {test_score[0]}, Test accuracy: {test_score[1]}')

8/8 [==============================] - 5s 594ms/step - loss: 0.1213 - accuracy: 1.0000
Test loss: 0.12129541486501694, Test accuracy: 1.0
